In [2]:
import numpy as np
from scipy.optimize import linprog

costs_1 = np.array([
    [30, 24, 11, 12, 25],
    [26, 4, 29, 20, 24],
    [27, 14, 14, 10, 18],
    [6, 14, 28, 8, 2]
])

supply_1 = np.array([21, 19, 15, 25])
demand_1 = np.array([15, 15, 15, 15, 20])

costs_2 = np.array([
    [5, 4, 20, 5, 14],
    [19, 20, 2, 4, 5],
    [9, 6, 8, 19, 1],
    [7, 15, 20, 6, 11]
])

capacities = np.array([
    [5, 20, 14, 25, 13],
    [19, 22, 15, 4, 12],
    [6, 8, 15, 5, 10],
    [14, 7, 2, 23, 20]
])

supply_2 = np.array([60, 50, 40, 20])
demand_2 = np.array([10, 50, 40, 50, 20])

In [4]:
import numpy as np
from scipy.optimize import linprog

def check_numeric_data(C, supply, demand, capacity=None):
    if not (np.issubdtype(C.dtype, np.number) and np.all(np.isfinite(C))):
        raise ValueError("Cost matrix 'C' must contain only numeric and finite values.")
    if not (np.issubdtype(supply.dtype, np.number) and np.all(np.isfinite(supply))):
        raise ValueError("Supply vector must contain only numeric and finite values.")
    if not (np.issubdtype(demand.dtype, np.number) and np.all(np.isfinite(demand))):
        raise ValueError("Demand vector must contain only numeric and finite values.")
    if capacity is not None:
        if not (np.issubdtype(capacity.dtype, np.number) and np.all(np.isfinite(capacity))):
            raise ValueError("Capacity matrix must contain only numeric and finite values.")

def validate_dimensions(C, supply, demand, capacity=None):
    check_numeric_data(C, supply, demand, capacity)
    if len(supply) != C.shape[0]:
        raise ValueError("Supply vector length must match the number of rows in cost matrix 'C'.")
    if len(demand) != C.shape[1]:
        raise ValueError("Demand vector length must match the number of columns in cost matrix 'C'.")
    if capacity is not None and capacity.shape != C.shape:
        raise ValueError("Capacity matrix dimensions must match those of cost matrix 'C'.")

def ensure_balanced_supply_demand(supply, demand):
    if not np.isclose(np.sum(supply), np.sum(demand)):
        raise ValueError("Total supply and demand must be equal.")

def adjust_imbalance(supply, demand, cost_matrix, capacity=None, high_value=1e6):
    supply_total = np.sum(supply)
    demand_total = np.sum(demand)

    if supply_total > demand_total:
        demand = np.append(demand, supply_total - demand_total)
        cost_matrix = np.hstack((cost_matrix, np.zeros((cost_matrix.shape[0], 1))))
        if capacity is not None:
            capacity = np.hstack((capacity, np.full((capacity.shape[0], 1), high_value)))
    elif demand_total > supply_total:
        supply = np.append(supply, demand_total - supply_total)
        cost_matrix = np.vstack((cost_matrix, np.zeros((1, cost_matrix.shape[1]))))
        if capacity is not None:
            capacity = np.vstack((capacity, np.full((1, capacity.shape[1]), high_value)))

    return supply, demand, cost_matrix, capacity

def transport_simple(C, supply, demand):
    rows, cols = C.shape
    cost_vector = C.flatten()

    equality_constraints = np.zeros((rows + cols, rows * cols))
    supply_demand_vector = np.concatenate((supply, demand))

    for i in range(rows):
        equality_constraints[i, i * cols:(i + 1) * cols] = 1

    for j in range(cols):
        equality_constraints[rows + j, j::cols] = 1

    bounds = [(0, None)] * (rows * cols)

    solution = linprog(cost_vector, A_eq=equality_constraints, b_eq=supply_demand_vector, bounds=bounds, method='highs')

    if solution.success:
        return solution.x.reshape(rows, cols), solution.fun
    raise ValueError("No feasible solution found for the transportation problem.")

def transport_with_capacity(C, capacity, supply, demand):
    rows, cols = C.shape
    cost_vector = C.flatten()

    equality_constraints = np.zeros((rows + cols, rows * cols))
    supply_demand_vector = np.concatenate((supply, demand))

    upper_bounds = np.eye(rows * cols)
    capacity_vector = capacity.flatten()

    for i in range(rows):
        equality_constraints[i, i * cols:(i + 1) * cols] = 1

    for j in range(cols):
        equality_constraints[rows + j, j::cols] = 1

    bounds = [(0, None)] * (rows * cols)

    solution = linprog(cost_vector, A_eq=equality_constraints, b_eq=supply_demand_vector, A_ub=upper_bounds, b_ub=capacity_vector, bounds=bounds, method='highs')

    if solution.success:
        return solution.x.reshape(rows, cols), solution.fun
    raise ValueError("No feasible solution found for the transportation problem with capacity constraints.")

def solve_transport_problem(C, supply, demand, use_capacity=False, capacity=None):
    try:
        validate_dimensions(C, supply, demand, capacity)
        ensure_balanced_supply_demand(supply, demand)

        if use_capacity:
            if capacity is None:
                raise ValueError("Capacity matrix must be provided when using capacity constraints.")
            supply, demand, C, capacity = adjust_imbalance(supply, demand, C, capacity)
            return transport_with_capacity(C, capacity, supply, demand)

        supply, demand, C, _ = adjust_imbalance(supply, demand, C)
        return transport_simple(C, supply, demand)
    except ValueError as error:
        return str(error)


In [7]:
solution, cost = solve_transport_problem(costs_1, supply_1, demand_1, use_capacity=False)

print("=" * 60)
print(" " * 15 + "Результати розв'язання транспортної задачі")
print("=" * 60)
print("\nМатриця розв'язку:")
print("-" * 60)
for row in solution:
    print(" | ".join(f"{cell:8.2f}" for cell in row))
print("-" * 60)

print("\nЗагальна вартість перевезення:")
print(f"💰 Вартість: {cost:,.2f} одиниць")
print("-" * 60)

print("\nДеталі постачань (постачальник -> споживач):")
for i, provider in enumerate([f'P{i}' for i in range(1, len(supply_1)+1)]):
    for j, customer in enumerate([f'P{i}' for i in range(1, len(demand_1)+1)]):
        if solution[i][j] > 0:  # Show only active connections
            print(f"🚚 Постачальник {provider} -> Споживач {customer}: {solution[i][j]:.2f} одиниць")

print("=" * 60)


               Результати розв'язання транспортної задачі

Матриця розв'язку:
------------------------------------------------------------
    0.00 |     0.00 |    15.00 |     6.00 |     0.00
    4.00 |    15.00 |     0.00 |     0.00 |     0.00
    0.00 |     0.00 |     0.00 |     9.00 |     6.00
   11.00 |     0.00 |     0.00 |     0.00 |    14.00
------------------------------------------------------------

Загальна вартість перевезення:
💰 Вартість: 693.00 одиниць
------------------------------------------------------------

Деталі постачань (постачальник -> споживач):
🚚 Постачальник P1 -> Споживач P3: 15.00 одиниць
🚚 Постачальник P1 -> Споживач P4: 6.00 одиниць
🚚 Постачальник P2 -> Споживач P1: 4.00 одиниць
🚚 Постачальник P2 -> Споживач P2: 15.00 одиниць
🚚 Постачальник P3 -> Споживач P4: 9.00 одиниць
🚚 Постачальник P3 -> Споживач P5: 6.00 одиниць
🚚 Постачальник P4 -> Споживач P1: 11.00 одиниць
🚚 Постачальник P4 -> Споживач P5: 14.00 одиниць


In [12]:
solution, cost = solve_transport_problem(costs_2, supply_2, demand_2, use_capacity=True, capacity=capacities)


print("=" * 60)
print(" " * 15 + "Результати розв'язання транспортної задачі")
print("=" * 60)
print("\nМатриця обмеження:")
print("-" * 60)
for row in capacities:
    print(" | ".join(f"{cell:8.2f}" for cell in row))
print("-" * 60)
print("\nМатриця розв'язку:")
print("-" * 60)
for row in solution:
    print(" | ".join(f"{cell:8.2f}" for cell in row))
print("-" * 60)

print("\nЗагальна вартість перевезення:")
print(f"💰 Вартість: {cost:,.2f} одиниць")
print("-" * 60)

print("\nДеталі постачань (постачальник -> споживач):")
for i, provider in enumerate([f'P{i}' for i in range(1, len(supply_2)+1)]):
    for j, customer in enumerate([f'P{i}' for i in range(1, len(demand_2)+1)]):
        if solution[i][j] > 0:
            print(f"🚚 Постачальник {provider} -> Споживач {customer}: {solution[i][j]:.2f} одиниць")

print("=" * 60)



               Результати розв'язання транспортної задачі

Матриця обмеження:
------------------------------------------------------------
    5.00 |    20.00 |    14.00 |    25.00 |    13.00
   19.00 |    22.00 |    15.00 |     4.00 |    12.00
    6.00 |     8.00 |    15.00 |     5.00 |    10.00
   14.00 |     7.00 |     2.00 |    23.00 |    20.00
------------------------------------------------------------

Матриця розв'язку:
------------------------------------------------------------
    5.00 |    20.00 |    10.00 |    25.00 |     0.00
    0.00 |    21.00 |    15.00 |     4.00 |    10.00
    5.00 |     8.00 |    15.00 |     2.00 |    10.00
    0.00 |     1.00 |     0.00 |    19.00 |     0.00
------------------------------------------------------------

Загальна вартість перевезення:
💰 Вартість: 1,336.00 одиниць
------------------------------------------------------------

Деталі постачань (постачальник -> споживач):
🚚 Постачальник P1 -> Споживач P1: 5.00 одиниць
🚚 Постачальник P1 -